# Import

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
from collections import defaultdict
from PIL import Image
import numpy as np
import cv2

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

# Dataset Info

In [ ]:

# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"

# 테스트 데이터의 경로와 정보를 가진 파일의 경로를 설정
testdata_dir = "./data/test"
testdata_info_file = "./data/test.csv"

import os
os.chdir('..')

# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기
train_data = pd.read_csv(traindata_info_file)

# 테스트 데이터
test_data = pd.read_csv(testdata_info_file)

# t-SNE

In [ ]:
# 이미지 데이터를 벡터화 (이미지 크기를 64x64로 통일했다고 가정)
def image_to_vector(img_path):
    if not os.path.exists(traindata_dir+'/'+img_path):
        print(f"File not found: {traindata_dir+'/'+img_path}")
        return None
    img = cv2.imread(traindata_dir+'/'+img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not load image: {traindata_dir+'/'+img_path}")
        return None
    img_resized = cv2.resize(img, (64, 64))  # 이미지 크기를 64x64로 조정
    return img_resized.flatten()  # 1D 벡터로 변환

# 모든 이미지 벡터화
def extract_features(df):
    features = []
    labels = []
    for img_path, label in zip(df['image_path'], df['target']):
        feature = image_to_vector(img_path)
        if feature is not None:  # 이미지 로드가 성공했을 경우에만 추가
            features.append(feature)
            labels.append(label)
    return np.array(features), np.array(labels)

# features와 labels 준비
features, labels = extract_features(train_data)

# features를 표준화
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# 클래스 별로 나누어 시각화
def visualize_with_tsne(features, labels, classes, perplexity=30, learning_rate=200, n_iter=1000):
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=learning_rate, n_iter=n_iter)
    reduced = tsne.fit_transform(features)
    
    for i in range(0, len(classes), 10):  # 50개씩 나누기
        plt.figure(figsize=(10, 10))
        subset_classes = classes[i:i+10]
        mask = np.isin(labels, subset_classes)
        reduced_subset = reduced[mask]
        labels_subset = labels[mask]
        sns.scatterplot(x=reduced_subset[:, 0], y=reduced_subset[:, 1], hue=labels_subset, palette='tab10', s=10)
        plt.title(f't-SNE Visualization for Classes {subset_classes[0]} to {subset_classes[-1]}')
        plt.show()

# 클래스 목록 생성
unique_classes = np.unique(labels)

# 시각화 실행
visualize_with_tsne(scaled_features, labels, unique_classes, perplexity=40, learning_rate=300, n_iter=1500)

# Autoencoder

In [ ]:

def image_to_vector(img_path, transform=None):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img_resized = cv2.resize(img, (64, 64))
    img_normalized = img_resized.astype(np.uint8)  # uint8 타입으로 유지

    if transform is not None:
        img_normalized = transform(image=img_normalized)['image']
    
    return img_normalized.flatten() / 255.0  # 변환 후에 정규화

# 모든 이미지 벡터화
def extract_features_from_csv(csv_file, is_train=False):
    df = pd.read_csv(csv_file)
    features = []
    labels = []
    
    # 공통 변환 설정
    common_transforms = [
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]

    # 훈련용 변환 설정
    transform = None
    if is_train:
        transform = A.Compose([
            A.Rotate(limit=10, p=0.5),
            A.Affine(scale=(0.8, 1.2), shear=(-10, 10), p=0.5),
            A.ElasticTransform(alpha=1, sigma=10, alpha_affine=None, p=0.5),
            # A.Erosion(kernel=(1, 2), p=0.5),
            # A.Dilation(kernel=(1, 2), p=0.5),
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.MotionBlur(blur_limit=(3, 7), p=0.5),
            A.CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=255, p=0.5),
            A.OneOf([
                # A.AutoContrast(p=0.5),
                A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5),
            ], p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5)
        ] + common_transforms)

    for img_path, label in zip(df['image_path'], df['target']):
        img_path_full = os.path.join('./data/train/', img_path)  # 경로 수정
        feature = image_to_vector(img_path_full, transform)
        if feature is not None:
            features.append(feature)
            labels.append(label)  # 클래스 레이블 추가
            
    return np.array(features), np.array(labels)

# 데이터 전처리
def preprocess_data(data):
    scaler = StandardScaler()
    return scaler.fit_transform(data)

# Autoencoder 모델 정의
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# CSV 파일 경로 설정
csv_file = './data/train.csv'
# 특징 추출 (훈련 데이터)
features, labels = extract_features_from_csv(csv_file, is_train=True)
# print(f"Number of features: {len(features)}")
# print(f"Number of labels: {len(labels)}")

# 데이터 전처리
processed_data = preprocess_data(features)
tensor_data = torch.FloatTensor(processed_data).cuda()  # CUDA로 이동

# Autoencoder 훈련
input_dim = tensor_data.shape[1]
model = Autoencoder(input_dim).cuda()  # CUDA로 이동
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(tensor_data)
    loss = criterion(output, tensor_data)
    loss.backward()
    optimizer.step()

# 원본 데이터와 재구성된 데이터 비교 함수 (클래스별)
def visualize_comparison_by_class(original_data, reconstructed_data, labels, num_images=5):
    unique_labels = np.unique(labels)
    for label in unique_labels:
        idx = np.where(labels == label)[0]  # 클래스에 해당하는 인덱스
        plt.figure(figsize=(15, 5))
        for i in range(min(num_images, len(idx))):
            # 원본 이미지
            ax = plt.subplot(2, num_images, i + 1)
            plt.imshow(original_data[idx[i]].detach().cpu().numpy().reshape(224, 224), cmap='gray')
            plt.title(f"Original: {label}")
            plt.axis('off')

            # 재구성된 이미지
            ax = plt.subplot(2, num_images, i + 1 + num_images)
            plt.imshow(reconstructed_data[idx[i]].detach().cpu().numpy().reshape(224, 224), cmap='gray')
            plt.title(f"Reconstructed: {label}")
            plt.axis('off')

        plt.show()

# 원본 데이터와 재구성된 데이터 비교
visualize_comparison_by_class(tensor_data, model(tensor_data).cpu(), labels)
